# 量子门与量子线路基础操作

In [3]:
# pylint: disable=W0104
import sys
sys.path.append("../")

import numpy as np
from quditop.gates import H, X, Y, Z, RX, RY, RZ, SX, SY, SZ, UMG
from quditop.circuit import Circuit

In [4]:
# Build a quantum circuit

dim = 2
n_qudits = 4

circ = Circuit(dim, n_qudits, gates=[
    H(dim).on(0),
    X(dim, i=0, j=1).on(0, [2,3]),
    Y(dim, i=0, j=1).on(1),
    Z(dim).on(1, 2),
    RX(dim, i=0, j=1, pr='param').on(3),
    RY(dim, i=0, j=1, pr=1.0).on(2, 3),
    X(dim).on(1, [0, 2, 3]),     # Default: i=0, j=1
    RY(dim, pr=2.0).on(3, [0, 1, 2])
])

print(circ)

Circuit(
  (gates): ModuleList(
    (0): H(2|0)
    (1): X(2 0 1|0 <-: 2 3)
    (2): Y(2 0 1|1)
    (3): Z(2 0 1|1 <-: 2)
    (4): RX(2 0 1 param|3)
    (5): RY(2 0 1 _param_|2 <-: 3)
    (6): X(2 0 1|1 <-: 0 2 3)
    (7): RY(2 0 1 _param_|3 <-: 0 1 2)
  )
)


In [5]:
# print the parameter names, where the `_param_` is default name.
circ.param_name

['param', '_param_', '_param_']

In [6]:
# We can add new gate(s) or circuit to old circuit easily.

circ += RZ(pr='pr_z').on(3)
circ.append(RX(pr=2.0).on(2))
circ += [X(dim).on(2), Y(dim).on(0, 3)]
circ += Circuit(dim, n_qudits, gates=[Z(dim).on(1), Z(dim).on(2)])

print(circ)

Circuit(
  (gates): ModuleList(
    (0): H(2|0)
    (1): X(2 0 1|0 <-: 2 3)
    (2): Y(2 0 1|1)
    (3): Z(2 0 1|1 <-: 2)
    (4): RX(2 0 1 param|3)
    (5): RY(2 0 1 _param_|2 <-: 3)
    (6): X(2 0 1|1 <-: 0 2 3)
    (7): RY(2 0 1 _param_|3 <-: 0 1 2)
    (8): RZ(2 0 1 pr_z|3)
    (9): RX(2 0 1 _param_|2)
    (10): X(2 0 1|2)
    (11): Y(2 0 1|0 <-: 3)
    (12): Z(2 0 1|1)
    (13): Z(2 0 1|2)
  )
)


In [7]:
# print the summary of circuit.

circ.summary()

==========================Circuit Summary==========================
|Total number of gates   : 14.                                    |
|Parameter gates         : 5.                                     |
|with 5 parameters are   :                                        |
|param, _param_, _param_, pr_z, _param_                         . |
|Number qudits of circuit: 4                                      |


In [8]:
# Get the quantum state of circuit.
print(f"quantum state in numpy.array:\n{circ.get_qs()}\n")
print(f"quantum state in ket:\n{circ.get_qs(ket=True)}")

quantum state in numpy.array:
[ 0.        +0.j          0.        +0.j         -0.45824748+0.25034174j
 -0.45824748+0.25034174j  0.        +0.j          0.        +0.j
  0.1607426 +0.29423735j  0.1607426 +0.29423735j  0.04210032-0.07706411j
  0.        +0.j          0.21969555+0.12002024j -0.17759523-0.19708435j
  0.12002023+0.06556735j  0.        +0.j          0.07706412-0.1410649j
  0.04295611+0.20663226j]

quantum state in ket:
-0.4582+0.2503j¦0010⟩
-0.4582+0.2503j¦0011⟩
0.1607+0.2942j¦0110⟩
0.1607+0.2942j¦0111⟩
0.0421-0.0771j¦1000⟩
0.2197+0.12j¦1010⟩
-0.1776-0.1971j¦1011⟩
0.12+0.0656j¦1100⟩
0.0771-0.1411j¦1110⟩
0.043+0.2066j¦1111⟩


In [9]:
# We can get the state of quantum with specific value.
default_value = 0.0
qs = circ.get_qs({'param': 1.0, 'pr_z': 2.0, '_param_':default_value})

print(f'n_qudits: {circ.n_qudits}')
print(f'qs:\n{qs}')

n_qudits: 4
qs:
[ 0.        +0.j          0.        +0.j          0.        +0.j
  0.        +0.j          0.        +0.j          0.        +0.j
  0.52217025+0.33528167j  0.52217025+0.33528167j  0.        +0.j
  0.        +0.j          0.        +0.j          0.        +0.j
  0.        +0.j          0.        +0.j          0.2852629 -0.18316522j
 -0.2852629 +0.18316522j]


In [10]:
# print the trainable paramters, this interface is herited from torch.nn.Module.
list(circ.parameters())

[Parameter containing:
 tensor([1.], requires_grad=True),
 Parameter containing:
 tensor([0.], requires_grad=True),
 Parameter containing:
 tensor([0.], requires_grad=True),
 Parameter containing:
 tensor([2.], requires_grad=True),
 Parameter containing:
 tensor([0.], requires_grad=True)]

In [11]:
# Convert the circuit to encoder, we can see that there's no `requires_grad` for parameters.
circ.as_encoder()
# You can also do it by method `circ.no_grad()`
list(circ.parameters())

[Parameter containing:
 tensor([1.]),
 Parameter containing:
 tensor([0.]),
 Parameter containing:
 tensor([0.]),
 Parameter containing:
 tensor([2.]),
 Parameter containing:
 tensor([0.])]

In [12]:
# Compare with `mindquantum`
# Here we build a circuit by mindquantum with the same parameters.

import mindquantum as mq
    
mq_circ = mq.Circuit([
    mq.H.on(0),
    mq.X.on(0, [2,3]),
    mq.Y.on(1),
    mq.Z.on(1, 2),
    mq.RX(1.0).on(3),
    mq.RY(0.0).on(2, 3),
    mq.X(1, [0, 2, 3]),
    mq.RY(0.0).on(3, [0, 1, 2]),
    mq.RZ(2.0).on(3),
    mq.RX(0.0).on(2),
    mq.X(2),
    mq.Y(0, 3),
    mq.Z(1),
    mq.Z(2)
])

mq_qs = mq_circ.get_qs()

print(f'n_qubit: {mq_circ.n_qubits}')
print(f'mindquantum qs:\n{mq_qs}')

n_qubit: 4
mindquantum qs:
[ 0.        +0.j          0.        +0.j         -0.        +0.j
 -0.        +0.j         -0.        +0.j         -0.        +0.j
  0.52217026+0.33528167j  0.52217026+0.33528167j  0.        +0.j
  0.        +0.j         -0.        +0.j         -0.        +0.j
 -0.        +0.j         -0.        +0.j          0.28526291-0.18316521j
 -0.28526291+0.18316521j]


In [13]:
# We can see that the QuditWorld circuit has the same output as MindQuantum

print(np.isclose(qs, mq_qs))

[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True]


In [14]:
# QuditWorld support qudits.
import torch

dim = 3
n_qudits = 4

circ2 = Circuit(dim, n_qudits, gates=[
    UMG(dim, mat=torch.randn(dim, dim)).on(2),
    UMG(dim, mat=torch.randn(dim, dim)).on(0),
    RX(dim, i=0, j=2, pr=1.0).on(2),
    RY(dim, i=1, j=2).on(3, 2),
    RZ(dim, i=2, j=0).on(3, 2),
])

print(f"circ2:\n{circ2}")
print(f"circ2 qs:\n{circ2.get_qs()}")

circ2:
Circuit(
  (gates): ModuleList(
    (0): UMG(3|2)
    (1): UMG(3|0)
    (2): RX(3 0 2 _param_|2)
    (3): RY(3 1 2 _param_|3 <-: 2)
    (4): RZ(3 2 0 _param_|3 <-: 2)
  )
)
circ2 qs:
[-2.5192971 +0.24041227j -0.06222615+0.00593814j -0.12079111+0.01152689j
  0.        +0.j          0.        +0.j          0.        +0.j
  0.        +0.j          0.        +0.j          0.        +0.j
  0.5883034 +0.3213916j   0.01453098+0.00793831j  0.02820701+0.01540956j
  0.        +0.j          0.        +0.j          0.        +0.j
  0.        +0.j          0.        +0.j          0.        +0.j
 -0.44007167+1.3762983j  -0.01086968+0.0339943j  -0.02109983+0.06598849j
  0.        +0.j          0.        +0.j          0.        +0.j
  0.        +0.j          0.        +0.j          0.        +0.j
  0.        +0.j          0.        +0.j          0.        +0.j
  0.        +0.j          0.        +0.j          0.        +0.j
  0.        +0.j          0.        +0.j          0.        +0.j
  0.  

In [18]:
# Note: In QuditWorld, we don't limit that the matrix must be unitary.
# We provide the `is_unitary()` method to check if the matrix of gate is unitary.

umg = UMG(dim, mat=torch.randn(dim, dim)).on(2)
print(f"umg.matrix:\n{umg.matrix()}")
print(f"is_unitary: {umg.is_unitary()}")

umg.matrix:
tensor([[ 0.2322+0.j, -0.0897+0.j,  0.6270+0.j],
        [ 1.9810+0.j, -0.5620+0.j,  0.1469+0.j],
        [-0.3233+0.j,  0.0664+0.j,  1.2227+0.j]])
is_unitary: False
